# ECS 171 Group Project - Group 10 - Reference ANN

In [3]:
import numpy as np
import pandas as pd

np.random.seed(171)

#Contains 81 features extracted from 21263 superconductors along with the critical temperature in the 82nd column
X_train = pd.read_csv('./../../Data/X_train.csv', header=None)
Y_train = pd.read_csv('./../../Data/Y_train.csv', header=None)

X_test  = pd.read_csv('./../../Data/X_test.csv', header=None)
Y_test  = pd.read_csv('./../../Data/Y_test.csv', header=None)

## Untuned Reference Neural Network

In [12]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense

# source: https://jmlb.github.io/ml/2017/03/20/CoeffDetermination_CustomMetric4Keras/
from tensorflow.keras import backend as K
# calculate R^2
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

nn = Sequential(
    [
        keras.Input(shape=X_train.shape[1]),
        layers.Dense(20, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(100, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(60, activation='relu'),
        layers.Dense(20, activation='relu'),
        layers.Dense(1)
    ]
)


nn.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
           loss="mean_squared_error",
           metrics=[keras.metrics.RootMeanSquaredError(
    name="root_mean_squared_error", dtype=None), coeff_determination])

NUM_EPOCHS = 215
history = nn.fit(X_train, Y_train, batch_size=64, epochs=NUM_EPOCHS, verbose=0)

pred = nn.predict(X_test)
results = nn.evaluate(X_test, Y_test)

print()
print("train  mse: " + str(history.history['loss'][NUM_EPOCHS-1]))
print("train rsme: " + str(history.history['root_mean_squared_error'][NUM_EPOCHS-1]))
print("train  R^2: " + str(history.history['coeff_determination'][NUM_EPOCHS-1]))
print()
print("test   mse: " + str(results[0]))
print("test  rmse: " + str(results[1]))
print("test   R^2: " + str(results[2]))

220/220 [==============================] - 0s 456us/step - loss: 186.2332 - root_mean_squared_error: 13.6467 - coeff_determination: 0.8296

train  mse: 113.8455810546875
train rsme: 10.669844627380371
train  R^2: 0.8991572856903076

test   mse: 186.23316955566406
test  rmse: 13.646727561950684
test   R^2: 0.8296017646789551


## Tuned Neural Network

In [7]:
# source: https://www.sicara.ai/blog/hyperparameter-tuning-keras-tuner
#         https://pythonprogramming.net/keras-tuner-optimizing-neural-network-tutorial/
#.        https://www.youtube.com/watch?v=vvC15l4CY1Q

from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential()
    model.add(keras.Input(shape=X_train.shape[1]))
    
    for i in range(hp.Int('units', min_value=1, max_value=8, step=1)):
        model.add(
            Dense(
                units=hp.Int('units', min_value=10, max_value=40, step=5),
                activation='relu'
            )
        )
    
    model.add(
        Dense(1, activation='linear')
    )

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float(
                'learning_rate',
                min_value=1e-4,
                max_value=1e-2,
                sampling='LOG',
                default=1e-3
            )
        ),
        loss='mean_squared_error',
        metrics=[keras.metrics.RootMeanSquaredError(name="root_mean_squared_error", dtype=None),
                 keras.metrics.MeanSquaredError()]
    )

    return model

SEED=1
MAX_TRIALS=100
EXECUTION_PER_TRIAL=1

tuner = RandomSearch(
    build_model,
    objective='mean_squared_error',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='random_search1',
    project_name='group10'
)

tuner.search(x=X_train,
             y=Y_train,
             epochs=215,
             batch_size=64,
             validation_data=(X_test,Y_test))

INFO:tensorflow:Reloading Oracle from existing project random_search1/group10/oracle.json
INFO:tensorflow:Reloading Tuner from random_search1/group10/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [13]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 574       
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 5

In [14]:
import math
tuned_model = tuner.get_best_models(num_models=1)
tuned_results = tuned_model[0].evaluate(X_test,Y_test)
print("\ntuned rmse: " + str(tuned_results[1]))

220/220 [==============================] - 0s 426us/step - loss: 232.7189 - root_mean_squared_error: 15.2551 - mean_squared_error: 232.7189

tuned rmse: 15.255128860473633
